# EDA Customer Loans In Finance

## Aim
1. Identify outliers, assess skewness, understand types of missing data, and explore correlations within the dataset.
2. Transform the data to ensure correct data types, promote normal distribution, and impute missing values.
3. Analyze the transformed loan data to identify emerging patterns and enhance loan risk management strategies.

In [ ]:
from db_utils import load_credentials, RDSDatabaseConnector
import matplotlib.pyplot as pyplot
from pandas.tseries.offsets import DateOffset


from format import DataFormat
import pandas as pd


from plotter import Plotter


from transform import DataFrameTransform


from db_utils import load_data


from info import DataFrameInfo


import numpy as np


import warnings


warnings.simplefilter(action='ignore', category=FutureWarning)



## Check percentage of missing columns and Shape

In [ ]:
df = load_data("loan_payments.csv")
info = DataFrameInfo(df)

print(info.get_shape())
print(info.percentage_null())

plt = Plotter(df)

plt.missing_nulls_vis()

In [ ]:
info.df.dtypes

# Formating Data to appropriate types based on held information

### Strings to Boolean

In [ ]:
Transformer = DataFormat(df)
# Convert n and y to bool values
Transformer.string_to_boolean('payment_plan')

### Strings to Date 

# String dates to date format
string_dates = ['last_credit_pull_date', 'next_payment_date',
                'last_payment_date', 'earliest_credit_line', 'issue_date']

Transformer.strings_to_dates(string_dates)

### String to number strip

**(Note)...**
We don't convert these columns : 'mths_since_last_record', 'mths_since_last_major_derog' to int since they include 0 months since last to tell recent entry and null for NO entry

In [ ]:
string_to_num_cols = ['term', 'employment_length']
numerical_cols = ['term', 'mths_since_last_record',
                  'mths_since_last_major_derog', 'mths_since_last_delinq', 'mths_since_last_record', 'delinq_2yrs']

# Month to integer
Transformer.extract_num_from_string(string_to_num_cols)

Transformer.numerical_cols(numerical_cols)

Transformer.to_int(['term', 'open_accounts', 'total_accounts',
                    'collections_12_mths_ex_med', 'delinq_2yrs', 'employment_length', 'funded_amount'])

Transformer.round_float('collection_recovery_fee', 2)

### Calculate Final Payment Date (needed for calculating projected loss later)

In [ ]:
Transformer.df['final_payment_date'] = Transformer.df.apply(
    lambda row: row['issue_date'] + DateOffset(months=row['term']), axis=1)

# Convert Categorical columns

**(Note)...**
#### Also Handle NMCAR values into bin categories
These are NMCAR because they tell that users do not have a negative loan history 

1. months_since_last_delinq 57.17
2. months_since_last_record 88.60
3. months_since_last_major_derog  86.17

In [ ]:
bins = [np.nan, 0, 36, 72, 108, 146]
bin_labels = ['Never', '1-3 Years',
              '4-6 Years', '7-9 Years', '10-12 Years']

bin_cols = ['months_since_last_delinq', 'months_since_last_record',
            'months_since_last_major_derog']

for col in bin_cols:
    Transformer.df[col] = pd.cut(Transformer.df[col], bins=bins,
                                 labels=bin_labels, right=True, include_lowest=True)
    Transformer.df[col] = Transformer.df[col].fillna('Never')


categories = ['grade', 'sub_grade', 'home_ownership',
              'verification_status', 'loan_status', 'purpose', 'employment_length']

Transformer.cols_to_categories(categories)
Transformer.cols_to_categories(bin_cols)

# Drop redundant columns

In [ ]:
# application and policy code have all same values across their whole columns
# out_prncp_inv/total_payment_inv is the same as out_prncp/total_payment
# next_payment_date doesn't seem useful to us since there are a lot missing values (possibly NMAR because suggest payed off loan)

drop_cols = ['application_type',
             'policy_code', 'out_prncp_inv', 'total_payment_inv', 'Unnamed: 0', 'id', 'next_payment_date'
             ]
Transformer.drop_cols(drop_cols)

# Impute missing values with appropriate methods

**(Note)...**

1. Missing *employment length* likely means unemployed so impute 0

In [ ]:
# Impute and further drop

t_form = DataFrameTransform(df)

# Assume When Employment length is missing it means they don't have a job
t_form.impute_zeros(['employment_length'])
# mean interest rate since its within normal dist
t_form.impute_median(['int_rate', 'loan_amount', 'funded_amount'])

# these rows have insignificant null vals
t_form.drop_null_rows(['last_payment_date', 'last_credit_pull_date'])

# Show cleaned and imputed data

In [ ]:
plt = Plotter(t_form.df)
plt.missing_nulls_vis()

# Save Data
RDSDatabaseConnector.save_to_csv(
    t_form.df, "formatted_loan_payments.csv")

# Measure Skew of Data

In [ ]:

# delinq appears more discrete than continuous so it's not included in skew analysis
numerical_cols = ['loan_amount',
                  'funded_amount_inv', 'int_rate', 'instalment', 'dti', 'annual_inc', 'total_payment', 'total_accounts', 'open_accounts', 'last_payment_amount']

plt.multi_hist_plot(numerical_cols)
Info.skew_data(numerical_cols)

# Data Skew Result
All numerical data displays a high positive skew, signifying a substantial overrepresentation of values by outliers. Additionally, both the mean and median significantly exceed the mode, indicating a pronounced asymmetry in the distribution.

In [ ]:
plt.multi_qq_plot(numerical_cols)

info = DataFrameInfo(t_form.df)

right_skewed_cols = ['annual_inc', 'total_payment', 'total_accounts',
                     'last_payment_amount', 'open_accounts', 'instalment', 'funded_amount_inv']


 * Noticing that a significant number of columns display outliers characterized by an almost exponential pattern. The skew values have now moved closer to 0, suggesting a more balanced distribution, making it suitable for implementing the Box-Cox transformation.

In [ ]:
t_form.box_cox_transform(
    ['loan_amount', 'instalment', 'int_rate', 'dti', 'funded_amount_inv', 'total_payment'])

# Reduce impact of outliers
log_transform_cols = ['annual_inc', 'total_accounts',
                      'open_accounts', 'last_payment_amount']

t_form.log_transform(log_transform_cols)

Info.skew_data(
    ['loan_amount', 'total_payment', 'instalment', 'int_rate', 'dti', 'funded_amount_inv', 'annual_inc', 'total_accounts', 'open_accounts', 'last_payment_amount'])


plt.multi_hist_plot(numerical_cols)

* Now the data exhibits a more normal distribution, making it suitable for linear regression, k-nearest neighbors, and SVM algorithms. Certain features, such as total_accounts and open_accounts, appear more discretely represented in the prepared dataset.

In [ ]:
plt.multi_qq_plot(numerical_cols)

# Measure of Correlation of data to understand which columns are redundant

In [ ]:
import statsmodels.formula.api as smf

plt.correlated_vars(numerical_cols)

model0 = smf.ols("funded_amount ~ funded_amount_inv", plt.df).fit()


def VIF(r2):
    return 1/(1-r2)


print(VIF(model0.rsquared))
RDSDatabaseConnector.save_to_csv(
    t_form.df, "transformed_data.csv")

# Current State of Loans

1. Summarizing the Current Recovery Rate Against Investor Funding

In [ ]:
df = pd.read_csv("formatted_payments.csv",
                 parse_dates=['issue_date', 'last_payment_date', 'final_payment_date'])

df['final_payment_date'] = df.apply(
    lambda row: row['issue_date'] + DateOffset(months=row['term']), axis=1)
df['total_to_pay'] = df['instalment'] * \
    df['term']

no_nill_invested = df['funded_amount_inv'] != 0

In [ ]:
# If null for funded_amount_inv (investors) use funded_amount (business)
original_df = df[~no_nill_invested]

df = df[no_nill_invested]

# Check If the loan is recovered
df['recovered_ratio'] = (
    df['total_payment'] / df['funded_amount_inv'])

original_df['recovered_ratio'] = (
    original_df['total_payment'] / df['funded_amount'])

# Recombine dataframes now with ratio
df = pd.concat([df, original_df], ignore_index=True)

# Convert to index for grouping
df['last_payment_date'] = pd.DatetimeIndex(df['last_payment_date'])

# Check if loan is recovered
df['recovered'] = df['recovered_ratio'].ge(1)
recovered_loans = df['recovered'] == True

# Filter by recovered loans
recovered_loan_df = df[recovered_loans]

percent_of_loans_recovered = round(len(recovered_loan_df) / len(df) * 100, 2)

print(f"Percent of Loans Currently Recovered: {percent_of_loans_recovered}%")

2. Projecting the Query 6 Months in the Future Based on Monthly Installments

In [ ]:
# Check what kind of further data can be recovered in 6 months' time

finished_term_mask = df['final_payment_date'] > df['last_payment_date']
future_df = df.loc[~recovered_loans & finished_term_mask].copy()


prediction_df = df.loc[~recovered_loans & finished_term_mask].copy()
prediction_recovered_df = df.loc[recovered_loans].copy()

prediction_df['recovered'] = 0
future_df['6m_future_total_payment'] = future_df['total_payment'] + \
    (future_df['instalment'] * 6)


future_df['recovered'] = (
    future_df['6m_future_total_payment'] / df['funded_amount_inv']).ge(1)


recovered_loans = future_df['recovered'] == True
recovered_loan_df_6m_future = future_df.loc[recovered_loans]


percent_of_loans_recovered = round(
    len(recovered_loan_df_6m_future) / len(df) * 100, 2)


print(f"Percent of Loans Currently Recovered: {percent_of_loans_recovered}%")

# Simulating and Graphing the future 6 month payments

3. An additional 15% of loans is assumed to be recovered based on installment payments. To visualize this insight, simulated data is created and graphed.

### Thought Process
The dataframe is initially separated into recovered and unrecovered slices to determine what percentage of the unrecovered dataframe could be paid off in 6 months.

The following steps were taken:

1. Set all the last payment dates to the latest date.
2. Slice only the rows with the latest date (initially, this is all the rows).
3. Add the total_payment and installment and increment the month by 1 to get simulated next month's payment.
4. The next iteration would select this last slice because it's the latest date and continue until 6 months.

(This process is similar to a window function per date.)

This approach was chosen to showcase the cumulative sum of the percentage change over months in recovered loans and visualize it in a graph.


In [ ]:
prediction_df = prediction_df[[
    'last_payment_date', 'total_payment', 'instalment', 'funded_amount_inv', 'member_id', 'recovered']]


def add_months(df, col, nmonths):
    df[col] = df[col] + DateOffset(months=nmonths)


def add_nums(df, col, col_2):
    df[col] = df[col] + df[col_2]


def copy_and_mask_df(df, mask):
    return df.loc[mask].copy()


# Simulate payments and dates
months_to_predict = 6

prediction_df.sort_values(
    by='last_payment_date', ascending=False, inplace=True)

# reset all values to be the latest date payment was made
prediction_df['last_payment_date'] = prediction_df['last_payment_date'].iloc[0]

# dataframe used for prediction
for _ in range(months_to_predict):
    latest_date = prediction_df['last_payment_date'].max()

    mask = (prediction_df['last_payment_date'] == latest_date) & (
        prediction_df['recovered'] == False)

    copy_df = copy_and_mask_df(prediction_df, mask)

    add_nums(copy_df, 'total_payment', 'instalment')
    add_months(copy_df, 'last_payment_date', 1)

    copy_df['recovered'] = (
        copy_df['total_payment'] / copy_df['funded_amount_inv']) >= 1

    prediction_df = pd.concat([prediction_df, copy_df], ignore_index=True)


min_date = prediction_df['last_payment_date'].min()
prediction_df = prediction_df[prediction_df['last_payment_date'] > min_date]
prediction_df = prediction_df[prediction_df['recovered'] == True]

In [ ]:
complete_df = pd.concat(
    [prediction_recovered_df, prediction_df], ignore_index=True)

# Group by month to count amount recovered that month
filtered_df = complete_df .set_index('last_payment_date').groupby(
    [pd.Grouper(freq="M")])['recovered'].count().reset_index()

# Sort by date
filtered_df.sort_values(
    by='last_payment_date', ascending=True, inplace=True)

# Cumulative sum the percentage recovered per month ( recovered count / total recovered + unrecovered count )
filtered_df['cumulative_sum'] = filtered_df['recovered'].apply(
    lambda x: x / (len(df)) * 100).cumsum()

filtered_df

* As observed, we have the anticipated final sum based on the previously calculated values:
    Percent of Loans Currently Recovered: 55.98% + Percent of Loans Currently Recovered: 15%

In [ ]:
mask_original_data = filtered_df['last_payment_date'] <= min_date
mask_future_data = filtered_df['last_payment_date'] >= min_date - \
    DateOffset(months=1)

original_df = filtered_df.loc[mask_original_data]
future_df = filtered_df.loc[mask_future_data]

pyplot.plot(original_df['last_payment_date'],
            original_df['cumulative_sum'], label='Recovered so far')
pyplot.plot(future_df['last_payment_date'],
            future_df['cumulative_sum'], label='Recovered in 6 months')

pyplot.legend()
pyplot.title('Current vs Future Recovered Loans')
pyplot.show()

# Calculating Loss

Via column *loan_status* on loans that are charged_off

In [ ]:
df = pd.read_csv("ormatted_payments.csv",
                 parse_dates=['issue_date', 'last_payment_date', 'final_payment_date'])


charged_off_mask = df['loan_status'] == 'Charged Off'



charged_off_df = df.loc[charged_off_mask].copy()



prc_charged_off = round((len(charged_off_df) / len(df)) * 100, 2)



print(f"Percentage Charged Off : {prc_charged_off}%")

In [ ]:
charged_off_df['expected_total_payment'] = charged_off_df['instalment'] * \
    charged_off_df['term']

total_paid = round(sum(charged_off_df['total_payment']), 2)

total_required = round(sum(charged_off_df['expected_total_payment']), 2)

prc_paid_off = round((total_paid / total_required) * 100, 2)

print(f"Paid Towards Charged Off: {total_paid}")
print(f"Required to be paid: {total_required}")
print(f"Percentage paid off: {prc_paid_off}%")

In [ ]:
# Instalments already include interest

missed_out_revenue = round(total_required - total_paid, 2)
prc_lost = round((missed_out_revenue / total_required) * 100, 2)

print(f"Missed out/ Increase in revenue: ${missed_out_revenue}")
print(f"Percentage Lost: {prc_lost}%")

### Possible Loss

In [ ]:
df['payments_made'] = df['total_payment'] / \
    df['instalment']
df['months_left_to_pay'] = df['term'] - \
    df['payments_made']

late_mask = df['loan_status'].str.contains('Late')

late_df = df.loc[late_mask].copy()

prc_behind = round((len(late_df) / len(df)) * 100, 2)

print(f"Percentage of Late Loans: {prc_behind}%")

In [ ]:
late_df['amount_left_to_pay'] = late_df['months_left_to_pay'] * \
    late_df['instalment']

projected_loss = round(late_df['amount_left_to_pay'].sum(), 2)

print(f"Projected Loss if Switched to Charged Off: ${projected_loss}")

In [ ]:
prc_late_and_charged_off = round(
    ((len(late_df) + len(charged_off_df)) / len(df)) * 100, 2)

print(
    f"Percentage of late and charged_off customers: {prc_late_and_charged_off}%")

# Indicator of Loss

Analyzing Potential Indicators of Loan Default

Identifying Data Points for Examination:

Highlight Data to Check
1. Loan Grade
2. Reason
3. DTI (Debt-to-Income Ratio)

Initially, the categorical columns will be converted to numerical representations to facilitate their application to the correlation matrix and Chi-Squared test.

In [ ]:
late_and_charged_off = pd.concat([late_df, charged_off_df])

categorical_cols = df.select_dtypes(
    exclude=np.number).columns.tolist()

_, b = pd.factorize(df[categorical_cols].values.ravel('F'))

df[categorical_cols] = df[categorical_cols].apply(
    lambda x: pd.Categorical(x, b).codes)


indicator_cols = ['grade', 'sub_grade', 'purpose',
                  'dti', 'delinq_2yrs', 'employment_length', 'last_payment_amount', 'loan_status']

plt = Plotter(df)
plt.correlated_vars(indicator_cols)

In [ ]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df['grade'], df['loan_status'])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic = {chi2}")
print(f"p-value = {p}")

# Loss prediction result


# Correlation Matrix
The correlation matrix indicates minimal correlation between the loan_status and any anticipated factors. Consequently, we won't be isolating a subset of charged-off customers for further exploration.

# Chi-Squared Test
The Chi-Squared Test reveals a remarkably high statistic and a p-value of 0, indicating a substantial difference among potential indicator columns.